In [1]:
import time
import torch.optim
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable
from datasets.dataset import CUBDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time

In [2]:
learning_rate = 1e-3
input_channels = 1
output_features = 6
epoch = 1
save_model_name = 'models/convolutional.pth'

In [3]:
dataset = CUBDataset(is_test=True)
dataloader = DataLoader(dataset=dataset, batch_size=10, shuffle=True)

In [4]:
conv = torch.hub.load('pytorch/vision:v0.9.0', 'vgg19', pretrained=True)
conv.eval()
conv.conv1 = nn.Conv2d(input_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
conv.fc = nn.Linear(in_features=512, out_features=output_features, bias=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv.parameters(), lr=learning_rate)

Using cache found in C:\Users\Vincent/.cache\torch\hub\pytorch_vision_v0.9.0


In [5]:
epoch_loss = list()

for epoch_number in range(epoch):
    running_loss, count, acc = 0., 0, 0.
    for img, label in dataloader:
        t = time.time()
        img = Variable(img)
        label = Variable(label).type(torch.long)
        output = conv(img)
        optimizer.zero_grad()
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        acc += (torch.max(output, dim=1)[1]==label).sum()
        count += img.size(0)
        print(count, time.time() - t)
    print(epoch_number, count, running_loss, int(acc)/count)
    epoch_loss.append(running_loss)

10 19.289747953414917
20 30.06543731689453
30 39.93372964859009
40 49.84142541885376
50 59.6364061832428
60 69.08192324638367


KeyboardInterrupt: 

In [ ]:
epoch_loss = list()

In [ ]:
epoch_loss.append((4,6,8))

In [ ]:
plt.plot(range(epoch), epoch_loss)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
torch.save(conv, save_model_name)

In [ ]:
count, acc = 0, 0.
for data in test_loader:
    img, label = data
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    output = conv(img)
    acc += (torch.max(output, dim=1)[1] == label).sum()
    count += img.size(0)
print(count, running_loss, int(acc)/count)